#https://huggingface.co/Komala/cnn_news_summary_model_trained_on_reduced_data
The fine-tuned t5-small is now available for Summarization in the Hugging face repository at the above link. This could be used directly means we can use this model for zero-shot learning for summerizing other text. which could be done after once we push the model to hub using the command trainer.push_to_hub()
Also, remember that we can access the tokenizer and model separately from our repository as per the need. Those are shown in the upcomming parts

## TODO Recording:

- First go to HuggingFace
- Under "Models" search for "T5 small" and show the model card page
- Go to Colab and log in (cloud.user@)
- Import this notebook from local machine
- Go to runtime and show that we are using the TPU
- After running pip install of libraries, please restart the kernel

Installing required huggingface libraries/modules

In [ ]:
pip install transformers datasets evaluate rouge_score accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=0599555727dd7f722c50a9af7e68afd5e01f55212275520917339fa45a1a4f40
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


Checking transformers version

In [ ]:
import transformers

transformers.__version__

'4.35.2'

## TODO Recording

- Go to the HuggingFace site
- Click on the Account on the top right -> Profile -> Settings -> Access Tokens
- Generate a new token and copy it
- Then run the cell below


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

https://huggingface.co/docs/datasets/v1.2.0/loading_datasets.html
Checking all datasets available in huggingface hub

In [ ]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

DatasetInfo(id='acronym_identification', author=None, sha='c3c245a18bbd57b1682b099e14460eebf154cbdf', last_modified=datetime.datetime(2023, 1, 25, 14, 18, 28, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=438, likes=17, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None)
DatasetInfo(id='ade_corpus_v2', author=None, sha='080cf99e1483008322d612c7262c04c8902fdbee', last_modified=datetime.datetime(2023, 6, 1, 14, 59, 53, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=1284, likes=20, paperswithcode_id=None, tags=['task_categories:text-classification', 'task_categories:token-classification', 'task_ids:core

Loading cnn daily mail dataset

In [ ]:
# from datasets import load_dataset
# # #Loading the cnn_dailymail dataset from the hugging face platform, Buts it is very large hence finetunning will take hours and
# # colab won allow us to use GPu for hours. Hence its take 3% of the data
# cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")
# #Viewing how the dataset look like
# cnn_news_summary_ds

from datasets import load_dataset
cnn_news_summry_ds = load_dataset("cnn_dailymail", "3.0.0")
cnn_news_summry_ds

Executing this will download the dataset and make it available to your notebook here on Colab. Here at the bottom, you can see what this dataset looks like.
DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

 It comprises of training, validation, and test data. There are about 287,000 records in the training data, 13,000 in the validation dataset, and 11,000 in the test dataset. The T5 transformer model is a very large model and you'll find that it takes a long time to fine-tune on this dataset. If you were to work with such a large dataset, fine-tuning the model might take hours and Colab doesn't really let you use the GPU for several hours in one go. Rather than working with this entire dataset, we'll work with a subset. We'll work with just 3% of the original data. Observe that I invoked load dataset once again on the CNN Daily Mail dataset, and I specify split = training data and 3% of the training data. Now, when the dataset is loaded, observe that we have just 8,613 rows from the training data.

Dataset dictionary can be seen. As the dataset is quite large, we would be taking a subset of training data only.

In [ ]:
from datasets import load_dataset
cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0", split = "train[:3%]")

cnn_news_summary_ds
# Dataset({
#     features: ['article', 'highlights', 'id'],
#     num_rows: 8613
# })

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8613
})

Subset data's dimensions can be checked

In [ ]:
cnn_news_summary_ds.shape

(8613, 3)

dataset description, features, homepage can be obtained
https://huggingface.co/docs/datasets/v1.2.0/exploring.html

In [ ]:
#print(cnn_news_summary_ds.description)
print(cnn_news_summary_ds.description)
#cnn_news_summary_ds.description
# CNN/DailyMail non-anonymized summarization dataset.
# There are two features:
#   - article: text of news article, used as the document to be summarized
#   - highlights: joined text of highlights with <s> and </s> around each
#     highlight, which is the target summary

CNN/DailyMail non-anonymized summarization dataset.

There are two features:
  - article: text of news article, used as the document to be summarized
  - highlights: joined text of highlights with <s> and </s> around each
    highlight, which is the target summary



In [ ]:
cnn_news_summary_ds.description

'CNN/DailyMail non-anonymized summarization dataset.\n\nThere are two features:\n  - article: text of news article, used as the document to be summarized\n  - highlights: joined text of highlights with <s> and </s> around each\n    highlight, which is the target summary\n'

In [ ]:
#Displayin the features of the dataset
cnn_news_summary_ds.features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [ ]:
#Printing the home page of the CNN daily mail dataset which shows the github page where the code oroginally sourced out
cnn_news_summary_ds.homepage
#https://github.com/abisee/cnn-dailymail

'https://github.com/abisee/cnn-dailymail'

#One row of example can be obtained

In [ ]:
cnn_news_summary_ds[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

#Dataset is split into train and test sets

In [ ]:
cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed = 42)

cnn_news_summary_ds
# DatasetDict({
#     train: Dataset({
#         features: ['article', 'highlights', 'id'],
#         num_rows: 6890
#     })
#     test: Dataset({
#         features: ['article', 'highlights', 'id'],
#         num_rows: 1723
#     })
# })

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Text cleaning function is defined

In [ ]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", "")
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example


Both train and test sets are cleaned using 'map' method

In [ ]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Taking a look at a raw example

In [ ]:
cnn_news_summary_ds["train"]["article"][7]

'(CNN)  -- Four-time defending champion Rafael Nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to Sweden\'s Robin Soderling at the French Open on Sunday. Soderling savors his unlikely triumph against Nadal in the French Open fourth round. Soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at Roland Garros and was a short-priced favorite to win a record fifth straight title. But Soderling never looked back after breezing through the first set of their fourth round match and closed out victory as Nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. The Spaniard tried to look on the positives after his shock defeat. "It\'s not a tragedy, losing here in Paris," he told the official tournament Web site www.rolandgarros.com . "It had to happen one day, and this is an excellent season for me. "Of course it\'s a bit sad, but I have to overcome this as quickly as possible. No one remembers defea

Now observing cleaned up version of the same raw data

In [ ]:
cleaned_cnn_news_summary_ds["train"]["article"][7]

"(cnn)   four-time defending champion rafael nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to sweden's robin soderling at the french open on sunday. soderling savors his unlikely triumph against nadal in the french open fourth round. soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at roland garros and was a short-priced favorite to win a record fifth straight title. but soderling never looked back after breezing through the first set of their fourth round match and closed out victory as nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. the spaniard tried to look on the positives after his shock defeat. it's not a tragedy, losing here in paris, he told the official tournament web site www.rolandgarros.com . it had to happen one day, and this is an excellent season for me. of course it's a bit sad, but i have to overcome this as quickly as possible. no one remembers defeats on the

Similarly,both raw and cleaned up version of highlights(summary) of same example can be seen

In [ ]:
cnn_news_summary_ds["train"]["highlights"][0]

'Maradona staying a clinic in Italy when tax authorities swooped .\nReports: Argentina coach owes millions in taxes from his time playing in Italy .\nMaradona says Serie A club should have paid the taxes .'

In [ ]:
cleaned_cnn_news_summary_ds["train"]["highlights"][0]

'maradona staying a clinic in italy when tax authorities swooped .reports: argentina coach owes millions in taxes from his time playing in italy .maradona says serie a club should have paid the taxes .'

taking a random article for summarising

In [ ]:
EXAMPLE_TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

In [ ]:
MODEL_NAME = "t5-small"

##https://huggingface.co/t5-small

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever . lancey, a 10-week-old puppy, was hand-delivered to them earlier this week . he's the most human of any dog we've ever had, the company says ."}]

Gives a warning (887 > 512).<br/>

Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors
[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever . lancey, a 10-week-old puppy, was hand-delivered to them earlier this week . he's the most human of any dog we've ever had, the company says ."}]  <br/>

Hence, We truncate the sequence length by setting truncation=True

In [ ]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)

summary_txt = summarizer(prefix + example_text)

summary_txt

[{'summary_text': 'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'}]

Gold/Reference summary is obtained

In [ ]:
ref_txt = cleaned_cnn_news_summary_ds["test"]["highlights"][EXAMPLE_TEXT_INDEX]

ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

Rouge metric is instantiated

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"

Rouge score with extra words in summary in comparison to reference(gold summary)

In [ ]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

Fewer extra words and better the score

In [ ]:
results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

when the rouge metrics are loaded, you can see what this evaluation metric returns. It returns rouge1, rouge2, rougeL, and rougeLsum scores for whatever data you parse in. RougeLsum is a variant of the rougeL metric that we've discussed earlier. Here, the longest common subsequence is computed between each pair of reference and candidate sentences, and a metric called Union LCS is computed. And this Union LCS is the rougeLsum. You can see the code that we'll use to compute rouge scores in the examples section. The rouge score is in the range 0 to 1. A rouge score close to 0 indicates poor similarity between candidate and reference, and a rouge score close to 1 indicates strong similarity between candidate and reference. With this in mind, let's see an example of how the rouge metric is computed. Here is the reference sentence on line 1, "the elephant was found near the river and everyone was glad." Then we have the first example, which has a lot of extra words such as turbulent and ecstatic. Example 2 is another candidate sentence with fewer extra words. We'll now compute the rouge scores between the two examples and the reference sentence and you can see what the results look like. Let's first compute the rouge score between the first example with a lot of extra words and the reference sentence. Here, the reference sentence and the candidate are very similar, even though the example, that is, the candidate sentence contains many extra words. So you can see the rouge scores are rather high: 0.83 for rouge1, 0.63 for rouge2, 0.83 for both rougeL and rougeLsum. Let's compute the rouge score for the second candidate and reference sentence. Example 2 contains fewer extra words. Here, you can see that the rouge1 score is higher, almost 0.87. It's 0.869. Rouge2 is a little lower at 0.57. RougeL and RougeLsum are also higher at almost 0.87. So now that you know how rouge scores work,

Coming back to our predicted summary
<br/>

let's take a look at the summary that was generated for the article about cloning the Labrador retriever. So this is the summary. Let's compute the rouge score of this summary against the original reference text, that is, the summary that is present in the dataset. Use_stemmer = True means that a Porter stemmer will be applied to the candidate and reference sentences before route scores are computed. The Porter stemmer will strip out suffixes of words so that words such as talk and talking will be considered to be the same word. The rouge score for our summary compared to the reference summary is only about 0.13, so it isn't a great score.

In [ ]:
summary_txt[0]["summary_text"]

'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'

Rouge score for that text summary is obtained

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

The rouge score is 0.131 is not a Great Score
{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

All article texts and summaries are defined in a list

In [ ]:
article_texts = cleaned_cnn_news_summary_ds["test"]["article"]

article_summaries = cleaned_cnn_news_summary_ds["test"]["highlights"]

Generating summaries (zero shot) for first 50 rows only as it will takes a very long time in summarizing whole set of examples <br/>

What is tqdm () in Python?<br/>
TQDM: Python Library to Monitor Code Progress (Updated 2023)
What Is tqdm in Python? Tqdm is a popular Python library that provides a simple and convenient way to add progress bars to loops and iterable objects.

In [ ]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [03:04<00:00,  3.68s/it]


Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.3260735008054134,
 'rouge2': 0.13887950380739084,
 'rougeL': 0.23956724610180005,
 'rougeLsum': 0.2383299157369655}

Here case by case rouge scores are obtained.

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.34146341463414637,
  0.38805970149253727,
  0.2608695652173913,
  0.28125,
  0.27848101265822783,
  0.18181818181818182,
  0.3855421686746988,
  0.24615384615384614,
  0.5974025974025975,
  0.3076923076923077,
  0.33766233766233766,
  0.3235294117647059,
  0.17821782178217824,
  0.1917808219178082,
  0.2278481012658228,
  0.3582089552238805,
  0.2702702702702703,
  0.3235294117647059,
  0.2682926829268293,
  0.32432432432432434,
  0.35443037974683544,
  0.28205128205128205,
  0.40540540540540543,
  0.10666666666666667,
  0.3116883116883117,
  0.36363636363636365,
  0.32558139534883723,
  0.5757575757575758,
  0.42857142857142855,
  0.24444444444444444,
  0.271604938271605,
  0.37288135593220334,
  0.3,
  0.13157894736842105,
  0.3820224719101123,
  0.44,
  0.4507042253521127,
  0.5217391304347826,
  0.647887323943662,
  0.2105263157894737,
  0.1038961038961039,
  0.2153846153846154,
  0.1590909090909091,
  0.3711340206185567,
  0.3548387096774194,
  0.2571428571428571,
  

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)
#(38, 12) index 38 has max rouge score and index 12 has least rouge score

(38, 12)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,firefighter jimmy senk went inside a burning a...,horia cretan became a hero for carrying boy fr...
1,nigeria's princess nikky onyeri has dedicated ...,health advocate from nigeria campaigns for bet...
2,troy davis has always maintained his innocence...,supreme court's latest ruling means troy davis...
3,"more than 11,500 public servants have been sus...",international transparency poll: businesses in...
4,human rights watch says the displaced sri lank...,"group: more than 280,000 displaced tamil civil..."


For the best rouge score, we can observe the predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

Actual Summary
the statue of kate moss will be displayed in the british museum in october . it is the largest gold statue built since ancient egypt . the statue will be part of a group of major sculptures .
Reference Summary
statue of kate moss billed as largest gold statue built since ancient egypt .called siren, the statue will be on display at the british museum .moss statue will be displayed in gallery housing ancient greek sculpture .


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Actual Summary
mobile computer chips have gotten so powerful and affordable, and it doesn't make much sense to ship a gadget with dedicated, stagnant functionality . developers have created 100,000 apps for the store at no cost to apple, which takes a cut of revenues .
Reference Summary
companies are opening app stores and bringing applications to tvs, netbooks and even cars .intel launched an app store for netbooks at the consumer electronics show this week .samsung will launch an app store later this year for hdtvs and blu-ray players .ford motor co. will make select apps available for new vehicles later in 2010 .


Video Transcript
#Understanding tokenizers<br/>
Selecting transcript lines in this section will navigate to timestamp in the video
Previously we generated summaries using the T5 model with zero-shot learning, that is, we didn't fine-tune the model before using it to generate summaries. We just used the pre-trained model directly. We'll now continue working with the T5 model, except that, we'll fine-tune the model on this CNN dataset that we have. Fine-tuning is an approach to transfer learning where we start with the weights of the pre-trained model and then train it on the dataset that we have. We'll see if the fine-tuned model produces better summaries than just using the pre-trained model. In order to be able to fine-tune this model, we need to understand in a little more detail the steps involved when we run a Hugging Face pipeline. The steps include first preparing and processing the inputs that you parse in, then running the model on those inputs to get predictions and then processing these outputs, these predictions so that they return to you in a form that you can understand. When we fine-tune our model, we'll have to perform each of these steps individually, starting with the first step, tokenization. This is where we pre-process and prepare the input text so that it's converted to a form that can be fed into our model for predictions. We access the tokenizer for our T5 Small model using the AutoTokenizer class. AutoTokenizer.from_pretrained and by parsing in the model name will give you the tokenizer for your model. Every NLP model in the Hugging Face library will have a different tokenizer. This will give us the tokenizer for our model. In order to see what kind of data we feed into our model, let's apply this tokenizer to this text, four-time defending champion. Observe that the result is in the form of input IDs. The tokenizer will break the input sentence into sub-words. For example, four-time will become two sub-words, defending might become two sub-words as well. And every sub-word has a unique ID. Those are the input IDs that you see. The attention mask tells the model what tokens to pay attention to. For example, if you had some text and then you add a bunch of padding after that text, well, the attention mask would tell the model to ignore the padding by having all 0s for the padded tokens. When you invoke the tokenizer in this form, it actually performs two separate operations. The first is tokenizer.tokenize, and this is what generates the sub-words or tokens from the input text. You can see four is a separate token, the hyphen is a separate token, then time, and that's why we get the six tokens. The second operation is to convert these tokens to IDs. So you can call tokenizer.convert_tokens_to_ids, parse in the input tokens, and we get the exact same result that we got earlier. These are the same tokens we got for the input text when we just invoked the tokenizer directly on the text.

Loading  a T5 tokenizer to process text and summary

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Tokenizer is applied and token ids can be seen

In [ ]:
tokenizer(["four-time defending champion"])

{'input_ids': [[662, 18, 715, 3, 20309, 6336, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
tokens = tokenizer.tokenize("four-time defending champion")

tokens

['▁four', '-', 'time', '▁', 'defending', '▁champion']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[662, 18, 715, 3, 20309, 6336]

#Loading T5 Model with AutoModelForSeq2SeqLM To load the pretrained model from Hugging Face
Video Transcript for the Below Parts <br/>

<b>Fine-tuning the T5 small model</b>
Just like we separately access the tokenizer to preprocess our data, let's separately load the model which we plan to fine-tune. And we do this using AutoModelForSeq2SeqLM. This is the class used to load language models with the encoder-decoder architecture, which is the case for our T5 model. We use the from_pretrained method, specify our model name, and this will give us the pre-trained model that we'll fine-tune. We'll now fine-tune this model on our CNN Daily Mail training data. Remember, we are working with a small sample of the original dataset. We'll use the tokenizer to preprocess the samples that we feed in to fine-tune the model. And we do this within the preprocess function. The input to the preprocess function are the examples that need to be preprocessed. Remember that every input text article needs to be prefixed with summarize:, which is why on line 6, for each document in the example articles we add the prefix in and those are our inputs. We then tokenize these inputs by invoking the tokenizer. We set the max length for number of tokens to be 1024. Any text which generates tokens greater than 1024 will be truncated. That's what truncation = True does. Tokenizing the input articles gives us the model inputs for fine-tuning. We then run the tokenizer on the highlights. This is on line 9. The highlights are the reference summaries that we'll use to train the model. The max length I set to 128 tokens, once again, truncation = True. We now have the original articles and the summaries from our training data. We set the labels key on the model inputs to be equal to the input IDs generated for the highlights, that is, labels["input_ids"] and we return the model inputs. We run this preprocessed function on our entire dataset, both the training data as well as the test data using the map function. Batched = True allows us to speed up the map function so that it processes multiple elements of the dataset at one go. Let's take a look at the features in our preprocessed data. We have the original article, the highlights, the ID, those come from the original dataset. We also have input IDs, attention mask, and labels. The input IDs, if you remember, are the numeric tokens fed into our model. The attention mask will indicate to the model what tokens to pay attention to. If there are padding tokens, the attention mask will ask the model to ignore those. And the labels are, of course, the input IDs or the highlights, that is, the reference summaries. In order to feed in our training data in batches to the model, we'll use a data collator. Data collators are objects that will form a batch by using a list of dataset elements as input. In order to create batches, data collators may apply some kind of processing like padding or apply some random data augmentation. We instantiate a data collator for our task by specifying the tokenizer and the model name. We are almost there. We are almost ready to fine-tune our model on the CNN Daily Mail training set that we have. But before that, I'm going to set up another Python function, and this one will compute rouge scores for the summaries generated by the model. Compute metrics will take in eval_pred, which is essentially a tuple of the predictions and the actual labels, that is, the actual highlights from the dataset. On line 3, we use tokenizer.batch_decode to get decoded values of predictions. The output generated by the model will be in the form of token IDs, but in order for us to understand the output, we need it in the form of strings. So this is what tokenizer.batch_decode does. On line 5, we look at the tokenized labels. Remember, the labels are the actual highlights from the dataset. Wherever the token IDs for the labels are equal to -100, we'll replace that with a padding token ID. The documentation doesn't say exactly why this is needed, but if you do not have this little line of code, you'll find computing metrics on your predictions will result in errors. After we perform this replacement, we use tokenizer.batch_decode to decode the labels as well and convert them to strings. Once we have the decoded predictions and decoded labels, we can compute rouge scores. Remember, rouge.compute takes in predictions and the reference values in the form of strings. Result is a dictionary that will hold the aggregated rouge scores for our predictions. We also compute the average length of our predictions. And this is the code on lines 10 and 11. We add the average length of predictions to the result as well, and finally, we return values in the result rounded off to four decimal places. We are ready to start training our pre-trained model. This is fine-tuning. Now, before we do that, we need to specify the arguments for running the training process. And this we do using sequence-to-sequence training arguments. Here we specify the output directory where the serialized model will be stored, the evaluation strategy in terms of epochs, the learning rate of the model, the batch sizes, the number of epochs of training you can see on line 9, I've set that to 3. Predict_with_generate, I've specified as True. This indicates whether the model should use generate to calculate generative metrics such as rouge. We of course want that to be true. Setting push_to_hub = True will push the model as well as the tokenizer to a repository on the Hugging Face hub. These training arguments that we've set up are used to customize the trainer that will actually perform training on the model. The trainer takes in a number of arguments, the model that needs to be fine-tuned, the training arguments, the train dataset, the evaluation dataset, the tokenizer for the input data, the data collator that will basically collect your input data in batches, and finally, the compute metrics, metrics that will be generated on the results. I then invoke the train method on the trainer. This is on line 25. Colab for some reason displays a red underline, but there's nothing really wrong with that code. You'll see that it runs through just fine. Go ahead and hit "Shift + Enter" and we'll fine-tune our model on the CNN Daily Mail dataset for three epochs. Now, this took a fairly long time to run. Training run for about 50 minutes to an hour on the single GPU that I have here on Colab.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

The preprocessing function you want to create needs to:

Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Use the keyword text_target argument when tokenizing labels.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["highlights"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

Applying the preprocessing function over the entire dataset by using 'map' method. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once

In [ ]:
tokenized_cnn_news_summary_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched = True)

tokenized_cnn_news_summary_ds

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1723
    })
})

Now create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
# Put the Write Token below HF login to continue executing the next code part


Defining our training hyperparameters in Seq2SeqTrainingArguments. The only required parameter is output_dir which specifies where to save your model. We will push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the ROUGE metric and save the training checkpoint.
Passing the training arguments to Seq2SeqTrainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
Calling train() to finetune your model.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "cnn_news_summary_model_trained_on_reduced_data",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds["train"],
    eval_dataset = tokenized_cnn_news_summary_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()
# "Token is required (write-access action) but no token found. You need"
#     174                 " to provide a token or be logged in to Hugging Face with"

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,1.623476,0.217600,0.093500,0.182800,0.183000,19.000000
2,1.920000,1.607144,0.216900,0.093200,0.182700,0.182700,19.000000
3,1.825800,1.603656,0.217100,0.093300,0.183000,0.183100,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1293, training_loss=1.859590865250991, metrics={'train_runtime': 1427.3117, 'train_samples_per_second': 14.482, 'train_steps_per_second': 0.906, 'total_flos': 5594970860814336.0, 'train_loss': 1.859590865250991, 'epoch': 3.0})

# https://huggingface.co/Komala/cnn_news_summary_model_trained_on_reduced_data
The fine-tuned t5-small is now available in the Hugging face repository at the above link. This could be used directly means we can use this model for zero-shot learning for summerizing other text. which could be done after once we push the model to hub using the command trainer.push_to_hub()
Also, remember that we can access the tokenizer and model separately from our repository as per the need. Those are shown in the upcomming parts

## TODO Recording:

- Immediately after training completes click on the link to the model on HuggingFace e.g. https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data
- Click on Model Card - should be nothing there
- Click on Files and Versions - should see the trained model files and some commits there
- Click on Training Metrics - that will take you to TensorBoard - show the graphs there (click on all tabs)

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1703725094.29816c8e791b.248.0:   0%|          | 0.00/7.55k [00:00<?, ?B/s]

'https://huggingface.co/Komala/cnn_news_summary_model_trained_on_reduced_data/tree/main/'

## TODO Recording:

- Now go to the model on HuggingFace
- Click on "Model Card" show that it has been generated
- Click on "Files and Versions" and show

In [ ]:
example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

The simplest way to try out your finetuned model for inference is to use it in a pipeline(). Instantiate a pipeline for summarization with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

prefix = "summarize: "

# summarizer = pipeline(
#     "summarization",
#     model = "loony-user/cnn_news_summary_model_trained_on_reduced_data",
#     truncation=True
# )
summarizer = pipeline(
    "summarization",
    model = "Komala/cnn_news_summary_model_trained_on_reduced_data",
    truncation=True
)
summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'summary_text': "a florida couple paid a california firm $155,000 to clone their beloved labrador retriever .a 10-week-old puppy dubbed lancey was hand-delivered to them earlier this week .edgar and nina otto were one of five families to bid and win a bioarts auction .sir lancelot's dna sample was sent to the sooam biotech research foundation ."}]

Video Transcript for the Below Parts <br/>
On the example text, make sure you add the prefix and let's see what the summary looks like. Here is the summary for the original article. The summary does capture the essence of the article, but whether it's better than the previous summary we got using zero-shot learning, well, it's hard to tell. We saw how we could use our fine-tuned pipeline directly, but we can also access the tokenizer and model separately from our repository. Here, I use AutoTokenizer.from_pretrained and I point to the model and tokenizer that we saved out on our repository. Then on line 5, I pass the example text along with the prefix to the tokenizer, return tensors = "pt" will return tensors in the PyTorch format and then I access the input IDs. The input IDs stored in the inputs variable are the form in which the model accepts data. This is what will pass into our model. Next, we use AutoModelForSeq2SeqLM.from_pretrained to load our model in. Again, specify the path to the repository where the model was saved, cloud-user/cnn_news_summary_model_trained_ on_reduced_data. We then get predictions from the model by calling model.generate and parse in the input tokens, that is, in the inputs variable. Model.generate will generate new text using the input text that we have specified. max_new_tokens are set to 100, which means the maximum number of tokens generated by this model will be 100.<b> do_sample = False means the model should not use sampling while generating text. It will use an algorithm called greedy decoding. While generating word sequences, greedy decoding will select the word that has the highest probability, it will act greedily</b>. The outputs variable that contains the predictions from our model is in the form of tokens, not in the human-readable format. So we need to call tokenizer.decode to actually decode those tokens and get the resulting summary as a string. And here we get the same summary that we got earlier when we used our fine-tuned model in a pipeline directly. We can now compute rouge metrics on summaries generated by our fine-tuned model. Let's look at the reference text for our example article first. So this is the reference summary for the article on the cloned Labrador. Now let's compute the rouge score for the summary generated by our fine-tuned model by comparing its summary with this reference text. Based on the rouge scores here, this summary is much better than the summary we got using zero-shot learning. When we use the pre-trained model directly to generate a summary, you can see the older scores here at the bottom of your screen and the current scores. Notice the rouge1 score is now 0.25, it was just 0.13 earlier. Also, notice the rougeL score is 0.153 now in the fine-tuned model and previously the rougeL score was 0.07. So clearly this model generates summaries that are far better than when we use just zero-shot learning. Rather than evaluate a fine-tuned model on just one article, let's generate summaries for the first 50 articles in our test dataset. We do this using a for loop in exactly the same way as before, and we'll compare this with our original pre-trained model. Let's compute the aggregated rouge scores on the 50 summaries that were generated by our fine-tuned model, and we'll compare these results with the model where we did not perform any fine-tuning. So here are the rouge scores. Let's compare them with the previous rouge scores that we received, and you can see these rouge scores are improved. Previously, the average rouge score for the first 50 summaries was 0.324, now it's 0.342. Rouge2 score went from 0.139 to 0.157. And you can see that both the rougeL and rougeLsum scores have also improved. Our fine-tuned model, even with just three epochs of training on a small training set is clearly better. Let's follow some steps that we've seen before. I'll get the unaggregated rouge scores for each of the 50 articles that were summarized by this model. Now that I have the unaggregated rouge scores, I'm going to extract the rougeLsum and compute the summary with the best rougeLsum score and the worst rougeLsum score. So the summary at Index 35 was the best and Index 3 was the worst. Let's now get a DataFrame of the predicted summaries from our fine-tuned model and the reference summaries from the actual dataset. Now that we have the predicted and actual summaries in this DataFrame format, we can compare the summary at Index 35 with the reference summary. Remember, this is the summary with the best rougeLsum score. And a quick glance at the actual summary as compared to the reference summary seems to indicate that the actual summary is pretty good. Now let's look at the worst index, Index 3. Let's look at the actual summary versus the reference summary, this had the worst rougeLsum score and you can see that the summaries are indeed very different. The model did not do a very good job with this one.
<br/> Transcript Ends

#Alternative way of summarising text(manual method)

In [ ]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")
tokenizer = AutoTokenizer.from_pretrained("Komala/cnn_news_summary_model_trained_on_reduced_data")

inputs = tokenizer(prefix + example_text, return_tensors = "pt", truncation=True).input_ids

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")
model = AutoModelForSeq2SeqLM.from_pretrained("Komala/cnn_news_summary_model_trained_on_reduced_data")

outputs = model.generate(inputs, max_new_tokens = 100, do_sample = False)

tokenizer.decode(outputs[0], skip_special_tokens = True)

'edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever.the 10-week-old puppy dubbed lancey was hand-delivered to them earlier this week.ottos are the first of six current clients to receive their clone.'

In [ ]:
ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = False)
result

{'rouge1': 0.2524271844660194,
 'rouge2': 0.039603960396039604,
 'rougeL': 0.1553398058252427,
 'rougeLsum': 0.1553398058252427}

Above Rouge Score is far better than the previous generated  one shown below for reference. Thus, the fine-tuned t5-small model works better than the zero shot usage of same t5-small model of hugging face.<br/>
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

output
output
{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

In [ ]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

  candidate = summarizer(prefix + text)

  candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [03:36<00:00,  4.33s/it]


Looks like zero shot untuned model is having slighly better performance than our fine tuned model.

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50], use_stemmer = False)
result_agg

{'rouge1': 0.3464107794902538,
 'rouge2': 0.1566639148509123,
 'rougeL': 0.25541178177491897,
 'rougeLsum': 0.25505353717660567}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

In [ ]:
result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)
#(35, 23)

(35, 23)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,bronx firefighter jimmy senk pulls 4-year-old ...,horia cretan became a hero for carrying boy fr...
1,nigeria's princess nikky onyeri speaks out to ...,health advocate from nigeria campaigns for bet...
2,troy davis has always maintained his innocence...,supreme court's latest ruling means troy davis...
3,"more than 11,500 public servants suspended or ...",international transparency poll: businesses in...
4,sri lankan government should release more than...,"group: more than 280,000 displaced tamil civil..."


Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(35, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(35, "Reference_summaries"))

Actual Summary
study: forests in the pacific northwest are dying twice as fast as they were 17 years ago .the data was gathered over a 50-year period at multiple sites in oregon, washington, california, arizona, colorado, new mexico and southwestern british columbia .
Reference Summary
forests in the pacific northwest are dying twice as fast as they were 17 years ago .scientists blame warming temperatures for the trend, according to a new study .data was gathered over a 50-year period at sites in the western u.s. and canada .scientists: study confirms the harmful effects of rising temperatures on ecosystems .


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(23, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(23, "Reference_summaries"))

Actual Summary
chandra levy, slain intern, had studied criminal justice in college, too .they mailed their findings to police chief in washington, d.c. they never heard back, but on saturday, the text and phone messages began to fly .a week ago, the police chief called his parents and told them a suspect would be arrested .
Reference Summary
group can't divulge whom it named as a suspect in chandra levy case .arrest warrant completely validates 15 months of work, professor says .professor recalls levy's mother being grateful that students kept spotlight on case .about 80 students at bauder college are members of crime club .
